In [1]:
import pandas as pd
import numpy as np
from auto_segment_FEMPO import BasicSegmenterEG_FEMPO
#dreload(auto_feature)
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import clone
boston = load_boston()
#print(boston.data.shape)

In [2]:
base_address = "./Notebooks/Datasets/"
dataSets = []
dataSets_temp = []
data_names = ['servo','abalone','ozone']
for i in range(0,len(data_names)):
    dataSets_temp.append(pd.read_csv(base_address+data_names[i]+'.csv'))
    temp_data = pd.read_csv(base_address+data_names[i]+'.csv')
    temp_data = pd.get_dummies(temp_data)
    temp_output = pd.DataFrame(temp_data['output'])
    temp_data.drop('output',axis=1,inplace=True)
    temp_data = pd.concat([temp_data, temp_output], axis=1)
    dataSets.append(temp_data)
# Boston Data
b_feat = pd.DataFrame(boston.data)
b_feat.columns = ['feat_0','feat_1','feat_2','feat_3','feat_4','feat_5','feat_6','feat_7','feat_8','feat_9','feat_10','feat_11','feat_12']
b_target = pd.DataFrame(boston.target)
b_target.columns = ['output']
b_data = pd.concat([b_feat,b_target],axis=1)
dataSets.append(b_data)

In [3]:
def get_result(model_,data_, frac):
    model = clone(model_)
    output = data_['output']
    data_.drop('output',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(data_, output, test_size=frac)
    model.fit(X_train,y_train)
    test_result = model.predict(X_test)
    return mean_squared_error(y_test,test_result), test_result, model

In [4]:
def get_result_iter(data, N_iterations, frac, models):
    #get_result(FS,temp_data)
    model_list = []
    Result_mses = []
    for i in range(0,len(models)):
        model_iter_results = []
        for j in range(0,N_iterations):
            temp_data = data.copy(deep=True)
            temp_rs, temp_pred, model = get_result(models[i],temp_data, frac)
            #print model.best_params_
            model_iter_results.append(temp_rs)
            model_list.append(model)
        Result_mses.append(sum(model_iter_results)/len(model_iter_results))
    return Result_mses, model_list

In [ ]:
Res_all_ds = []
model_f = []
for i in range(0,len(dataSets)):
    t_data = dataSets[i].copy(deep=True)
    N_iterations = 10
    g_frac = 0.10
    # Parameters I want to sweep on.
    parameters = {
        'n_votes': [1,3,5,10],
        'init_sample_percentage':[0.2,0.5,1]
    }
    # Default params
    FEMPO = BasicSegmenterEG_FEMPO(**{'cxpb': 0.5,
 'indpb': 0.2,
#  'init_sample_percentage': 0.5,
 'mutpb': 0.5,
 'n': 10,
 'n_population': 20,
 'ngen': 10,
 'statistics': False,
#  'test_size': 0.25,
 'tournsize': 3,
 'base_estimator': DecisionTreeRegressor(max_depth=5),
#  'n_votes' : 10
})
    
    #grid_model = RandomizedSearchCV(FS, parameters, verbose=True, scoring="mean_squared_error",n_iter=100)
    grid_model = GridSearchCV(FEMPO, parameters, verbose=True, scoring="mean_squared_error", cv = 3, n_jobs= 1 )
    g_models = []
    g_models.append(grid_model)
#     g_models.append(Ba)
    #g_models.append(linear_model.LassoCV(n_alphas=100))
    Result_t, model_list = get_result_iter(t_data, N_iterations, g_frac, g_models)
    Res_all_ds.append(Result_t)
    model_f.append(model_list)
dataSet_names = ['Servo','Abalone','Ozone','Boston-housing']
model_names = ['FEMPO']
Result_test_bench = pd.DataFrame(Res_all_ds)
Result_test_bench.columns = model_names
Result_test_bench.set_index([dataSet_names],inplace=True)
Result_test_bench

In [27]:
Result_test_bench

,FEMPO
Servo,0.143792
Abalone,4.872011
Ozone,18.674435
Boston-housing,15.642114


In [28]:
for m in model_f:
    print m[0].best_params_
#     print m

{'n_votes': 10, 'init_sample_percentage': 0.5}
{'n_votes': 10, 'init_sample_percentage': 0.5}
{'n_votes': 10, 'init_sample_percentage': 0.5}
{'n_votes': 5, 'init_sample_percentage': 1}


[GridSearchCV(cv=3, error_score='raise',
        estimator=BasicSegmenterEG_FEMPO(base_estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
             crossover_func=<function cxTwoPoint at 0x107C1DF0>, cxpb=0.5,
             indpb=0.2, init_sample_percentage=0.2, mutpb=0.5, n=10,
             n_population=20, n_votes=1, ngen=10, statistics=False,
             test_size=0.2, tournsize=3),
        fit_params={}, iid=True, loss_func=None, n_jobs=1,
        param_grid={'n_votes': [1, 3, 5, 10], 'init_sample_percentage': [0.2, 0.5, 1]},
        pre_dispatch='2*n_jobs', refit=True, score_func=None,
        scoring='mean_squared_error', verbose=True)]